In [ ]:
from __init__ import *

In [ ]:
ws, hs = read_ws_hs("../output/mf/", SETTING["model_mode"], SETTING["model_data_dicts"])

In [ ]:
distance_function = kraft.information.get_icd

In [ ]:
size = 800

layout = {"height": size, "width": size}

In [ ]:
factors = hs[0].index

factor_labels = factors.to_numpy()

factor_name = factors.name

In [ ]:
w_node_x_nodes = []

for i, w in enumerate(ws):

    w = w.to_numpy()

    node_x_node = kraft.array.apply_function_on_slices_from_2_matrices(
        w, w, 1, distance_function,
    )

    w_node_x_nodes.append(node_x_node)

    kraft.plot.plot_heat_map(
        node_x_node,
        factor_labels,
        factor_labels,
        factor_name,
        factor_name,
        layout={"title": {"text": "W {} Distance".format(i)}, **layout},
    )

In [ ]:
h_node_x_nodes = []

for i, h in enumerate(hs):

    h = h.to_numpy()

    node_x_node = kraft.array.apply_function_on_slices_from_2_matrices(
        h, h, 0, distance_function,
    )

    h_node_x_nodes.append(node_x_node)

    kraft.plot.plot_heat_map(
        node_x_node,
        factor_labels,
        factor_labels,
        factor_name,
        factor_name,
        layout={"title": {"text": "H {} Distance".format(i)}, **layout},
    )

In [ ]:
node_x_node = np.median(np.asarray(w_node_x_nodes + h_node_x_nodes), axis=0)

kraft.plot.plot_heat_map(
    node_x_node,
    factor_labels,
    factor_labels,
    factor_name,
    factor_name,
    layout={"title": {"text": "W H Distance"}, **layout},
)

In [ ]:
node_marker_size = 24

if 1 < len(SETTING["mf_rs"]):

    rs = np.asarray(tuple(int(factor.split(maxsplit=1)[1][0]) for factor in factors))

    node_marker_size *= 2 ** (1 - kraft.array.normalize(rs, "0-1"))

In [ ]:
directory_path = "../output/gps_map/"

kraft.path.path(directory_path)

In [ ]:
for w_h_str in ("w", "h"):

    if w_h_str == "w":

        point_x_node = pd.concat(ws)

    else:

        point_x_node = pd.concat(hs, axis=1).T

    point_x_node **= SETTING["{}_gps_map_node_power".format(w_h_str)]

    gps_map = kraft.gps_map.GPSMap(node_x_node, point_x_node)

    kraft.gps_map.write(
        "{}{}_gps_map.pickle.gz".format(directory_path, w_h_str), gps_map
    )

    gps_map.plot(
        node_trace={"marker": {"size": node_marker_size}},
        point_trace=SETTING["{}_gps_map_point_trace".format(w_h_str)],
        file_path="{}{}_gps_map.html".format(directory_path, w_h_str),
    )